### TP 1 EX 1
Grupo 5:
     
     Breno Fernando Guerra Marrão A97768
     
     Tales André Rovaris Machado A96314


## Inicialização
Usamos as bibliotecas pysmt para resovler o problema de alocação proposto 

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT


## Transformando CFA em um ciclo while

```Python
0:{ z == 0 and y >= 0 }
1: while (while y != 0:):
2:    if y % 2: 
        x = 2*x
        y = y/2
        if overflow:
5:           OVERFLOW   
3:     else:
        y = y-1
        z = z+x
                     
4: stop
```

## Transições

O estado inicial é caracterizado pelo seguinte predicado:

$$
\mathit{z} = 0 \wedge {pc} = 0  \wedge {y}\ge 0
$$

As transições possíveis no FOTS são caracterizadas pelo seguinte predicado:
$$
\begin{array}{c}
(\mathit{pc} = 0 \wedge \mathit{pc}' = 1 \wedge x' = x \wedge y' = y \wedge z' = z)\\
\vee\\
(\mathit{pc} = 1 \wedge \mathit{pc}' = 2 \wedge y \mod 2 = 0 \wedge x' = x \wedge y' = y \wedge z' = z) \\
\vee\\
(\mathit{pc} = 2 \wedge \mathit{pc}' = 1 \wedge x' \ge x \wedge x' = x * 2 \wedge y' = y \div 2 \wedge z' = z) \\
\vee\\
(\mathit{pc} = 2 \wedge \mathit{pc}' = 5 \wedge x > x * 2 \wedge x' = x * 2 \wedge y' = y \div 2 \wedge z' = z) \\
\vee\\
(\mathit{pc} = 5 \wedge \mathit{pc}' = 5 \wedge x' = x \wedge y' = y \div 2 \wedge z' = z) \\
\vee\\
(\mathit{pc} = 1 \wedge \mathit{pc}' = 3 \wedge  y \mod 2 = 1 \wedge  x' = x \wedge y' = y \div 2 \wedge z' = z) \\
\vee\\
(\mathit{pc} = 3 \wedge \mathit{pc}' = 1 \wedge  x' = x \wedge y' = y - 1 \wedge z' = z + x) \\
\vee\\
(\mathit{pc} = 1 \wedge \mathit{pc}' = 4 \wedge y = 0 \wedge  x' = x \wedge y' = y - 1 \wedge z' = z + x) \\
\vee\\
(\mathit{pc} = 4 \wedge \mathit{pc}' = 4  \wedge  x' = x \wedge y' = y \wedge z' = z) \\
\vee\\
\end{array}
$$



In [5]:

def declare(i,n):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)
    state['x'] = Symbol('x'+str(i),BVType(n))
    state['y'] = Symbol('y'+str(i),BVType(n))
    state['z'] = Symbol('z'+str(i),INT)
    return state



def init(state,a,b,n):
    
    return And(Equals(state['x'],BV(a,n)),
               Equals(state['y'],BV(b,n)),
               Equals(state['z'],Int(0)),
               Equals(state['pc'],Int(0)))

def trans(curr,prox,bv2,n):
    bv1 = BVOne(n)
    t0 = And(Equals(curr['pc'],Int(0)),
             Equals(prox['pc'],Int(1)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z'])) 
    
    t1 = And(Equals(curr['pc'],Int(1)),
             Equals(BVAdd(curr['y'],bv1),BVXor(curr['y'],bv1)),
             Equals(prox['pc'],Int(2)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    t2 = And(Equals(curr['pc'],Int(2)),
             Equals(prox['pc'],Int(1)),
             BVUGE(prox['x'],curr['x']),
             Equals(BVMul(curr['x'],bv2),prox['x']),
             Equals(BVUDiv(curr['y'],bv2),prox['y']),
             Equals(curr['z'],prox['z']))
    
    te = And(Equals(curr['pc'],Int(2)),
             Equals(prox['pc'],Int(5)),
             BVUGT(curr['x'],BVMul(curr['x'],bv2)),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    te2 = And(Equals(curr['pc'],Int(5)),
             Equals(prox['pc'],Int(5)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    t3 = And(Equals(curr['pc'],Int(1)),
             Equals(BVSub(curr['y'],bv1),BVXor(curr['y'],bv1)),
             Equals(prox['pc'],Int(3)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    t4 = And(Equals(curr['pc'],Int(3)),
             Equals(prox['pc'],Int(1)),
             Equals(curr['x'],prox['x']),
             Equals(BVSub(curr['y'],BVOne(n)),prox['y']),
             Equals(curr['z']+BVToNatural(curr['x']),prox['z']))
    
    t5 = And(Equals(curr['pc'],Int(1)),
             Equals(prox['pc'],Int(4)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']),
             Equals(curr['y'],BVZero(n)))
    
    t6 = And(Equals(curr['pc'],Int(4)),
             Equals(prox['pc'],Int(4)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    return Or(t1,t2,t3,t0,t4,t5,t6,te,te2)


def gera_traco(declare,init,trans,a,b,n,k):

    with Solver(name="z3") as s:
        
        bv2 = BV(2,n)
        
        trace = [declare(i,n) for i in range(k)]
        
        s.add_assertion(init(trace[0],a,b,n))
        
        for i in range(k-1):
            s.add_assertion(trans(trace[i],trace[i+1],bv2,n))
        
        if s.solve():
            for i in range(k):
                print("Passo", i)
                for v in trace[i]:
                    print(v,"=",s.get_value(trace[i][v]))
                if i>1:
                    if s.get_value(trace[i]["pc"]) == s.get_value(trace[i-1]["pc"]):
                        if s.get_value(trace[i]["pc"]) == Int(4):
                            print("Multiplicação efetuada com sucesso e valor :", s.get_value(trace[i]["z"]))
                            return
                        else:
                            print("OVERFLOW")
                            return

gera_traco(declare,init,trans,5,2,5,30)


Passo 0
pc = 0
x = 5_5
y = 2_5
z = 0
Passo 1
pc = 1
x = 5_5
y = 2_5
z = 0
Passo 2
pc = 2
x = 5_5
y = 2_5
z = 0
Passo 3
pc = 1
x = 10_5
y = 1_5
z = 0
Passo 4
pc = 3
x = 10_5
y = 1_5
z = 0
Passo 5
pc = 1
x = 10_5
y = 0_5
z = 10
Passo 6
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 7
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 8
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 9
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 10
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 11
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 12
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 13
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 14
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 15
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 16
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 17
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 18
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 19
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 20
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 21
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 22
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 23
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 24
pc = 4
x = 10_5
y = 0_5
z = 10
Passo 25
pc = 4
x 

## IDEIA : COLOCAR VARIOS GERATRACO COM EXEMPLOS Q ULTRAPASSA O NUMERO DE BITS O Q ACONTECE ... ETC , FALTOU NO ULTIMO RELATORIO